In [17]:
import pandas as pd
from glob import glob
import itertools
from collections import Counter
import numpy
import math
import matplotlib.pylab as plt
from urllib.parse import urlparse

In [18]:
# clean url
def urlclean(url):
    try:
        url = urlparse(url).hostname
        if url.replace('.','').isdigit(): return 'none'
        else:
            if len(url.split('.')) >=2 :
                if url[-6:]=='com.cn': return '.'.join(url.split('.')[-3:])
                return '.'.join(url.split('.')[-2:])
    except:
        return 'none'

In [ ]:
files=glob('./allusers/*')

column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,cont_classify_id,cont_type_id,acce_url'.split(',')
percentage=[]
for f in files:
    data=pd.read_csv(f,names=column_list)
    data=data[['site_name','acce_url']]
    data['acce_url']=data['acce_url'].map(lambda x: urlclean(x))
    data=data[data['acce_url']!='none']
#     if (len(data))&(len(data[data['site_name']!='其他']))>0:
    if len(data)>0:
        proportion=len(data[data['site_name']!='其他'])/len(data)
        percentage.append(proportion) 
        
df=pd.DataFrame(percentage)
df.to_csv('./percentage.csv')

In [46]:
files=glob('./date/*')
column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,\
app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,\
cont_classify_id,cont_type_id,acce_url'.split(',')

for f in files:
    data=pd.read_csv(f,names=column_list,usecols=['site_name','acce_url'])
    data['acce_url']=data['acce_url'].map(lambda x: urlclean(x))
    data=data[data['acce_url']!='none']
    
    if len(data)>0:
        proportion=len(data[data['site_name']!='其他'])/len(data)
        percentage.append(proportion) 
        
print(percentage)
print(sum(percentage)/len(percentage))

[0.9159049360146252, 0.0, 1.0, 0.6731182795698925, 0.986228813559322, 0.9309832522960562, 0.9823313940960999, 0.9032036298638801, 0.6031567080045096, 0.6268656716417911, 0.6708126036484245, 1.0, 0.8300433923806172, 0.8310420921162658]
0.7824064837993916


In [44]:
files=glob('/Users/xuhuimin/Documents/lab/labdata/beijingmobile/userexample/*')

column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,\
app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,\
cont_classify_id,cont_type_id,acce_url'.split(',')
percentage=[]
for f in files:
    data=pd.read_csv(f,names=column_list)
    data=data[['site_name','acce_url']]
    data['acce_url']=data['acce_url'].map(lambda x: urlclean(x))
    data=data[data['acce_url']!='none']
#     if (len(data))&(len(data[data['site_name']!='其他']))>0:
    if len(data)>0:
        proportion=len(data[data['site_name']!='其他'])/len(data)
        percentage.append(proportion) 
        
print(percentage)
print(sum(percentage)/len(percentage))

[0.9159049360146252, 0.0, 1.0, 0.6731182795698925, 0.986228813559322, 0.9309832522960562, 0.9823313940960999, 0.9032036298638801, 0.6031567080045096, 0.6268656716417911, 0.6708126036484245, 1.0]
0.7743837740578834


In [48]:
# seperate the date-files into user-files
files=glob('./date/*')

column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,\
app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,\
cont_classify_id,cont_type_id,acce_url'.split(',')

for i in files:
    data=pd.read_csv(i,names=column_list,usecols=['user_id','site_name','start_time'])    
    users=data['user_id'].drop_duplicates()
    groups=data.groupby('user_id')
    
    for user in users:
        df=groups.get_group(user)
        with open('./dateexample/allusers/' + str(user) +'.csv', 'a+') as f:
            df.to_csv(f,header=False)

In [3]:
# first put all the unique nodes together, second all pair combinations together,third pair combinations for every user
files=glob('./dateexample/allusers/*')
for f in files:
    user=f.split('/')[-1].split('.')[0]
    userdata=pd.read_csv(f,names=['user_id','start_time','site_name'])
    userdata=userdata.drop(['start_time'],axis=1)
    userdata=userdata.drop_duplicates()
    userdata=userdata[userdata['site_name']!='其他']
    
    userdata.to_csv('./dateexample/node.csv',header=False,mode='a+')
    
    busi_list=userdata['site_name'].values
    pair=list(itertools.combinations(busi_list,2))
    data=pd.DataFrame(pair)
    data.to_csv('./dateexample/pair.csv',header=False,mode='a+')
    
    with open('./dateexample/pair/' + str(user) +'.csv', 'a+') as f:
        data.to_csv(f,header=False)

In [4]:
# observed values
site_obs=pd.read_csv('./dateexample/pair.csv',names=['node1','node2'],index_col=0)
site_obs['pair']=site_obs['node1'].astype('str')+' '+site_obs['node2'].astype('str')
counterdict=Counter(site_obs['pair'].values)
counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','site_obs'])
counterdict.to_csv('./dateexample/observed_value.csv')

In [8]:
# expected values
for i in range (10): # reconstruct random network for 10 times
    name='site_exp'+str(i)
    name=pd.read_csv('./dateexample/node.csv',index_col=0,names=['user_id','site_name'])# read data
    numpy.random.shuffle(name['site_name'].values) # shuffle the data
    users=name['user_id'].drop_duplicates()
    groups=name.groupby('user_id')
    
    path='./dateexample/expected_value/'
    f=path+'site_exp'+str(i)+'.csv'

    for user in users:
        userdata=groups.get_group(user)
        userdata=userdata.drop_duplicates()

        site_list=userdata['site_name'].values
        pair=list(itertools.combinations(site_list,2))
        site_exp=pd.DataFrame(pair)
        site_exp.to_csv(f,header=0,mode='a+')

    site_exp=pd.read_csv(f,names=['node1','node2'])   
    site_exp['pair']=site_exp['node1'].astype('str')+' '+site_exp['node2'].astype('str')
    counterdict=Counter(site_exp['pair'].values)
    counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','site_obs'])
    counterdict.to_csv(f,header=0,mode='w')

In [9]:
# combine observed value and expected value
files=glob('./dateexample/expected_value/*')

for f in files:
    name=f.split('/')[-1].split('.')[0]
    data=pd.read_csv(f,names=['pair','count'])
    diction=dict(data[['pair','count']].values)
    z_score=pd.read_csv('./dateexample/observed_value.csv',index_col=0)
    z_score[name]=z_score['pair'].apply(lambda x: diction[x]
                                                    if x in diction
                                                    else numpy.nan)
    z_score.to_csv('./dateexample/observed_value.csv')

In [11]:
# z_score
z_score=pd.read_csv('./dateexample/observed_value.csv',index_col=0)
z_score=z_score.fillna(0)
mean=z_score.iloc[:,[x for x in range(2,5)]].mean(axis=1)
std=z_score.iloc[:,[x for x in range(2,5)]].std(axis=1)
z_score['z_score']=(z_score['site_obs']-mean)/std
z_score.to_csv('./dateexample/observed_value.csv')

In [17]:
df=pd.read_csv('./dateexample/observed_value.csv',index_col=0)
df=df[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

In [20]:
df=df.dropna(axis=0,how='any')

In [21]:
df.sort_values(ascending=False,by=['z_score'])[['pair','z_score']]

,pair,z_score
763,小米手机官网（xiaomi.net） 小米手机官网（xiaomi.com）,1712.420898
491,腾讯网 雅虎开发者网络,1098.290642
2064,DoubleClick数字广告 AdMob,775.381412
5043,360安全中心 360导航,666.550886
9000,360安全中心 拍拍网,650.096403
2450,支付宝 QQ空间图片服务,642.013499
387,360手机助手 奇虎360图片,594.821058
2321,数据专家 百度网盟推广,589.111478
5747,3G手机门户 3G门户,588.319924
116,360手机助手 百度,580.600000


In [23]:
# put z_score into every user-file
z_score=pd.read_csv('./dateexample/observed_value.csv',usecols=['pair','z_score'])
diction=dict(z_score.values)

files=glob('./dateexample/pair/*')

for f in files:
    data=pd.read_csv(f,names=['node1','node2'])
    data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
    data['z_score']=data['pair'].apply(lambda x: diction[x]
                                  if x in diction
                                   else np.nan)
    data.to_csv(f)

In [24]:
# four categories group by z_score
files=glob('./dateexample/pair/*')
lista=[]
listb=[]
listc=[]
for f in files:
    data=pd.read_csv(f)
    if len(data['z_score'])>0:
        user_id=f.split('/')[-1].split('.')[0]
        lista.append(user_id)
        a=numpy.array(data['z_score'].values)
        median=numpy.median(a)
        listb.append(median)
        pertentile_10=numpy.percentile(a,10)
        listc.append(pertentile_10)
df=pd.DataFrame([lista,listb,listc]).T
df.columns=['user_id','median','pertentile_10']
df.to_csv('./dateexample/zscore_category.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4387: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


In [26]:
# add category

df=pd.read_csv('./dateexample/zscore_category.csv',index_col=0)
df=df[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

high_low=df[(df['median']>df['median'].median())&(df['pertentile_10']>0)].index
df.ix[high_low,'category']='high_low'


low_low=df[(df['median']<=df['median'].median())&(df['pertentile_10']>0)].index
df.ix[low_low,'category']='low_low'

low_high=df[(df['median']<=df['median'].median())&(df['pertentile_10']<=0)].index
df.ix[low_high,'category']='low_high'

high_high=df[(df['median']>df['median'].median())&(df['pertentile_10']<=0)].index
df.ix[high_high,'category']='high_high'

df=df[['user_id','median','pertentile_10','category']]
df.to_csv('./dateexample/zscore_category.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [27]:
# fetch the valid information sex age fees brand terminal_price to zscore_category
columns = 'user_id，sex，age，occupation_name，education_name，user_opentime，brand_name，call_duration_m，gprs_flow，\
call_fee，gprs_fee，databusiness_fee，order_name，brand_chn，model_chn，screensize，operation_sys，terminal_price，\
dept_county_name，dept_name'.split('，')

basename=pd.read_csv('/Users/xuhuimin/Documents/lab/labdata/beijingmobile/basename.csv', encoding='gbk', names=columns)
basename=basename.drop_duplicates()
basename=basename.fillna(0)
basename['fees']=basename['call_fee']+basename['gprs_fee']+basename['databusiness_fee']

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df=pd.read_csv('./dateexample/zscore_category.csv',index_col=0)

names=['sex','age','brand_chn','terminal_price']
for n in names:
    diction=dict(basename[['user_id',n]].values)
    df[n]=df['user_id'].apply(lambda x: diction[x]
                                 if x in diction
                                 else numpy.nan)
    
# diction=dict(basename[['user_id','fees']].values)
# df['fees']=df['user_id'].apply(lambda x: diction[x]
#                              if x in diction
#                              else numpy.nan)

df.to_csv('./dateexample/zscore_category.csv')

In [30]:
# fetch the valid information unique business to zscore_cstegory
unique_busi=pd.read_csv('./dateexample/node.csv',index_col=0)
unique_busi.columns=['user_id','node']
unique_busi=unique_busi.groupby('user_id')['node'].nunique()

df=pd.read_csv('./dateexample/zscore_category.csv',index_col=0)

diction=dict(zip(unique_busi.index,unique_busi.values))
df['unique_busi']=df['user_id'].apply(lambda x: diction[x]
                             if x in diction
                             else numpy.nan)

df.to_csv('./dateexample/zscore_category.csv')

In [31]:
df

,user_id,median,pertentile_10,category,sex,age,brand_chn,terminal_price,unique_busi
0,37924550,8.408746,-15.158350,low_high,女,0.0,华为,1888.0,14
1,45285150,59.797320,19.943529,high_low,女,0.0,三星,1245.0,7
2,34026069,30.582053,-9.154320,high_high,女,0.0,联想,399.0,9
3,44047180,144.780392,7.041914,high_low,女,0.0,联想,816.0,5
4,31702228,37.197383,8.140240,high_low,女,0.0,夏新,999.0,9
5,2150681,27.103608,-9.903468,high_high,男,53.0,三星,2798.0,8
6,52920083,24.953325,-39.724880,high_high,女,0.0,诺基亚,629.0,6
7,30959534,10.901623,-0.992792,low_high,男,27.0,中国移动,456.0,13
8,30369557,1.154701,-4.016057,low_high,女,0.0,华为,0.0,6
9,46345630,30.331791,8.272035,high_low,不详,42.0,三星,2899.0,11


In [65]:
data=pd.read_csv('./dateexample/pair/46450066.csv')